In [2]:
# imports
from scipy.io import loadmat
import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

In [ ]:
# load data
matfile_baseline = loadmat('../data/raw/Ach-AT/00_0315_ach-at_0.mat')
matfile_ach = loadmat('../data/raw/Ach-AT/00_0315_ach-at_1.mat')
matfile_at = loadmat('../data/raw/Ach-AT/00_0315_ach-at_2.mat')

In [ ]:
def make_windows(matfile, window_size = 6000):
    # retrieve the relevant data from the dictionaries
    df = pd.DataFrame(matfile['filt_data'].copy())
    df = df.T
    time = pd.DataFrame(matfile['filt_t'].copy())
    time = time.T

    # set up the time column
    time.columns = ['t']
    time['t'] = time['t'] - 60000 # make time start at 0
    time['window_id'] = time['t'] / window_size # n second windows
    time['window_id'] = time['window_id'].astype(int)

    # join the electrode signals with time
    df = pd.concat([time, df], axis=1)

    return df

In [ ]:
matfile_baseline['config']

In [ ]:
df_base = make_windows(matfile_baseline)
df_ach = make_windows(matfile_ach)
df_at = make_windows(matfile_at)

In [ ]:
cols = ['t','window_id',4,27,34,57]
df_base[cols]

In [ ]:
# train on four electrodes - choose 4,27,34 and 57 (the centre-most ones)
df_base_4 = df_base[['t','window_id',4,27,34,57]]
df_base_4.loc[:,'y'] = 0
df_ach_4 = df_ach[['t','window_id',4,27,34,57]]
df_ach_4.loc[:,'y'] = 1
df_at_4 = df_at[['t','window_id',4,27,34,57]]
df_at_4.loc[:,'y'] = 2
# this could be done more efficiently, build into the function


df_4 = pd.concat([df_base_4, df_ach_4, df_at_4], axis=0, ignore_index=True)
y_4 = df_4[['window_id','y']]
df_4 = df_4.drop(columns = ['y'])
# y_4.to_hdf('0_0315_4_electrodes_y.h5', key = 'y', complevel = 9)

In [ ]:
y_4

In [ ]:
# extract minimal features and see how long it takes
X_minimal = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=MinimalFCParameters(), n_jobs=8)

In [ ]:
# extract more features
X_eff = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=EfficientFCParameters(), n_jobs=8)

In [ ]:
X_eff

In [ ]:
X_minimal.to_hdf('0_0315_4_electrodes_min.h5', key = 'features', complevel = 9)

In [ ]:
X_eff.to_hdf('0_0315_4_electrodes_eff.h5',key = 'features', complevel = 9)

In [3]:
df = pd.read_hdf('../data/processed/ach_at_hex_6000.h5')

In [4]:
df = df[['id',0]]

In [7]:
X_eff = extract_features(df, column_id='id', default_fc_parameters=EfficientFCParameters(), n_jobs=8)

Feature Extraction:   0%|          | 0/40 [00:41<?, ?it/s]Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/proce

KeyboardInterrupt: 

  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 39, in _function_with_partly_reduce
    results = list(itertools.chain.from_iterable(results))
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/feature_extraction/extraction.py", line 390, in _do_extraction_on_chunk
    return list(_f())
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/feature_extraction/extraction.py", line 390, in _do_extraction_on_chunk
    return list(_f())
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/feature_extraction/extraction.py", line 376, in _f
    result = func(x, param=parameter_list)
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 39, in _function_with_partly_r